In [1]:
import os
from dotenv import load_dotenv
from typing import TypedDict, Optional, Annotated, Sequence
import operator

from langgraph.graph import StateGraph, END, START
from langgraph.prebuilt import create_react_agent
from langchain_openai import ChatOpenAI
from langchain_core.messages import BaseMessage, HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate
from function import price_tool, human_fallback

In [5]:
# --- 0) ENV ---
load_dotenv()

# --- 1) State 정의 ---
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]
    next: Optional[str]

# --- 2) LLM ---
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# --- 3) Supervisor ---
members = ["price_agent", "consultation_agent", "human_fallback","recommendation_agent"]

supervisor_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a supervisor managing a perfume specialist team.

Your team members:
- price_agent: Handles perfume price/cost inquiries (has access to Naver Shopping API)
- consultation_agent: Handles general perfume advice 
- human_fallback: Handles non-perfume topics or unclear questions
- recommendation_agent: Handles personalized perfume recommendations using ML model and similarity search
     
Routing Rules:
- If query is about perfume price/cost (가격, 최저가, 얼마, price, cost, 구매, 판매), choose "price_agent"
- If query is perfume-related advice 향수간의 차이점, choose "consultation_agent"  
- If query is NOT about perfumes or is unclear/vague, choose "human_fallback"
- If query asks for perfume recommendations based on preferences/occasions (추천, 어울리는, 맞는, 좋은, recommend), choose "recommendation_agent"       

Respond with ONLY the agent name."""),
    ("placeholder", "{messages}"),
])

def supervisor_node(state: AgentState) -> dict:
    """Supervisor 노드"""
    chain = supervisor_prompt | llm
    result = chain.invoke(state)
    
    next_agent = result.content.strip()
    if next_agent not in members:
        next_agent = "human_fallback"  # 기본값
    
    return {"next": next_agent}

# --- 5) Recommendation Agent (ML 기반 추천) ---
recommendation_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a perfume recommendation specialist using advanced ML models.
    
Your capabilities:
- Use perfume_recommendation_tool to find personalized recommendations based on user preferences
- Analyze user's taste, occasion, season, and style preferences
- Provide detailed explanations of why certain perfumes match their needs
- Always respond in Korean with enthusiasm and expertise

When users ask for recommendations:
1. Use the perfume_recommendation_tool to get ML-based suggestions
2. Format the results beautifully with detailed explanations
3. Ask follow-up questions to refine recommendations if needed"""),
    ("placeholder", "{messages}"),
])

recommendation_agent = create_react_agent(
    llm,
    [perfume_recommendation_tool],
    prompt=recommendation_prompt
)

# --- 4) Price Agent (도구 포함) ---
price_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a perfume price specialist assistant.
    
When users ask about perfume prices:
1. Use the price_tool to search for current prices
2. Always respond in Korean
3. Format results nicely with emojis and clear information
4. Be helpful and friendly
    
If you can't find price information, politely explain and suggest alternative searches."""),
    ("placeholder", "{messages}"),
])

price_agent = create_react_agent(
    llm, 
    [price_tool],
    prompt=price_prompt
)

# --- 5) Consultation Agent (일반 상담) ---
consultation_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a knowledgeable and friendly perfume consultant.
    
Your expertise includes:
- Perfume recommendations based on preferences, occasions, seasons
- Fragrance families and notes explanation
- Perfume wearing tips and application advice
- Brand and fragrance history knowledge

Always respond in Korean with:
- Warmth and professionalism
- Helpful and detailed advice
- Relevant examples and suggestions
- Encouraging tone

If questions are too vague, politely ask for more specific information to provide better recommendations."""),
    ("placeholder", "{messages}"),
])

consultation_agent = create_react_agent(
    llm,
    [],  # 도구 없음
    prompt=consultation_prompt
)

# --- 6) Human Fallback Node ---
def human_fallback_node(state: AgentState) -> dict:
    """향수와 관련 없거나 불명확한 질문 처리"""
    # 마지막 메시지에서 사용자 입력 추출
    messages = state.get("messages", [])
    user_input = ""
    for msg in reversed(messages):
        if isinstance(msg, HumanMessage):
            user_input = msg.content
            break
    
    # human_fallback 함수 호출 (state dict 형태로 전달)
    fallback_result = human_fallback({"input": user_input})
    
    # 시스템 메시지로 응답 생성
    response_msg = HumanMessage(content=fallback_result)
    
    return {"messages": [response_msg]}

# --- 7) 에이전트 호출 래퍼 ---
def price_agent_node(state: AgentState) -> dict:
    """Price agent 호출"""
    result = price_agent.invoke(state)
    return {"messages": result["messages"]}

def consultation_agent_node(state: AgentState) -> dict:
    """Consultation agent 호출"""
    result = consultation_agent.invoke(state)
    return {"messages": result["messages"]}

def recommendation_agent_node(state: AgentState) -> dict:
    """Recommendation agent 호출"""
    result = recommendation_agent.invoke(state)
    return {"messages": result["messages"]}

# --- 8) Graph 구성 ---
workflow = StateGraph(AgentState)

# 노드 추가
workflow.add_node("supervisor", supervisor_node)
workflow.add_node("price_agent", price_agent_node)
workflow.add_node("recommendation_agent", recommendation_agent_node)
workflow.add_node("consultation_agent", consultation_agent_node)
workflow.add_node("human_fallback", human_fallback_node)

# 조건부 엣지: supervisor → agents
workflow.add_conditional_edges(
    "supervisor",
    lambda x: x["next"],
    {
        "price_agent": "price_agent",
        "consultation_agent": "consultation_agent",
        "recommendation_agent": "recommendation_agent",
        "human_fallback": "human_fallback",
    }
)

# 각 에이전트에서 END로
workflow.add_edge("price_agent", END)
workflow.add_edge("consultation_agent", END)
workflow.add_edge("human_fallback", END)
workflow.add_edge("recommendation_agent", END)

# 시작점
workflow.set_entry_point("supervisor")

# 컴파일
app = workflow.compile()

NameError: name 'perfume_recommendation_tool' is not defined

In [6]:
# --- 9) 실행 테스트 ---

query = "샤넬 함수중에 웜코튼같은 따듯한향 10만원대로 추천해줘"
print(f"\n[Query] {query}\n")

for step in app.stream({"messages": [HumanMessage(content=query)]}, stream_mode="updates"):
    for node, state in step.items():
        print(f"--- Node: {node} ---")
        for k, v in state.items():
            print(f"{k}: {v}\n")



[Query] 샤넬 함수중에 웜코튼같은 따듯한향 10만원대로 추천해줘



NameError: name 'app' is not defined

In [ ]:
from langchain_core.messages import HumanMessage, AIMessage

query = "20만원대 향수 추천좀 가격과함께 알려줄래"

result = app.invoke({"messages": [HumanMessage(content=query)]})

# 전체 state 확인
print("=== Raw State ===")
print(result)

# 마지막 AI 응답만 추출
msgs = result.get("messages", [])
last_ai = next((m for m in reversed(msgs) if isinstance(m, AIMessage)), None)
if last_ai:
    print("\n=== Final Answer ===")
    print(last_ai.content)


=== Raw State ===
{'messages': [HumanMessage(content='20만원대 향수 추천좀 가격과함께 알려줄래', additional_kwargs={}, response_metadata={}, id='2d74fb16-24b3-48e6-9cf8-cc78361d169a'), HumanMessage(content='20만원대 향수 추천좀 가격과함께 알려줄래', additional_kwargs={}, response_metadata={}, id='2d74fb16-24b3-48e6-9cf8-cc78361d169a'), AIMessage(content='안녕하세요! 20만원대의 향수를 찾고 계시군요. 여러 가지 멋진 선택지가 있습니다. 몇 가지 추천드릴게요.\n\n1. **조말론 런던 - 피오니 앤 블러쉬 스웨이드 (Peony & Blush Suede)**\n   - 가격: 약 18만원\n   - 특징: 부드러운 피오니와 사과의 조화가 매력적인 플로럴 계열의 향수입니다. 우아하고 여성스러운 느낌을 주며, 데일리로 사용하기 좋습니다.\n\n2. **디올 - 미스 디올 블루밍 부케 (Miss Dior Blooming Bouquet)**\n   - 가격: 약 19만원\n   - 특징: 상큼한 과일과 플로럴 노트가 어우러져 사랑스럽고 경쾌한 느낌을 줍니다. 봄과 여름에 특히 잘 어울리는 향수입니다.\n\n3. **겐조 - 플라워바이겐조 (Flower by Kenzo)**\n   - 가격: 약 17만원\n   - 특징: 파우더리한 플로럴 향으로, 독특한 매력을 지니고 있습니다. 특별한 날이나 저녁 외출에 잘 어울립니다.\n\n4. **샤넬 - 샹스 오 땅드레 (Chance Eau Tendre)**\n   - 가격: 약 20만원\n   - 특징: 상큼하고 부드러운 과일과 플로럴 노트가 조화를 이루며, 경쾌한 느낌을 줍니다. 데일리로 사용하기에 적합합니다.\n\n이 외에도 다양한 향수가 있으니, 어떤 향을 선호하시는지, 또는 어떤 상황에서 사용하고 싶으신